# Notebook 05 — FMV Forecasting (Stage 6)

## Verizon Cell Tower Lease Fair Market Value Estimation

This notebook implements **multi-year FMV forecasting** with scenario-based projections and a
**Lock-In vs Renegotiate decision framework** for the Verizon cell-tower lease portfolio.

We combine a **panel-regression model** (macro drivers × region fixed effects) with
**three scenario paths** (Bull / Base / Bear) to project every site’s FMV out to 2036.
The gap between contractual rent and projected FMV then feeds a **decision matrix** that
classifies each lease as *RENEGOTIATE_NOW*, *LOCK_IN*, or *MONITOR*.

### Pipeline Overview

| Section | Component | Description |
|---------|-----------|-------------|
| **6.1** | Setup & Data Load | Load FMV predictions from Stage 3–4 or fall back to synthetic data |
| **6.2** | Regional De-Escalated Rent Index | Construct time-series of de-escalated rents by metro region |
| **6.3A** | Panel Regression | OLS panel model with region fixed effects and macro variables |
| **6.3C** | Scenario Projections | Bull / Base / Bear FMV forecasts for 10-year horizon |
| **6.4** | Decision Matrix | Lock-In vs Renegotiate classification for every site |
| **6.5** | Portfolio Summary | Aggregate forecast metrics and overpayment trajectory |
| **6.6** | Save Outputs | Export `fmv_forecasts.parquet` |

### Inputs
- `../data/processed/fmv_predictions.parquet` (preferred — output of Notebook 03)
- `../data/synthetic/synthetic_5k_sites.parquet` (fallback — raw synthetic data)

### Outputs
- `../data/processed/fmv_forecasts.parquet` — site-level forecasts with scenarios and recommendations

### Key Columns in Output
| Column | Description |
|--------|-------------|
| `predicted_fmv_today` | Point-estimate FMV from Stage 3 model |
| `fmv_base_5yr / 10yr` | Base-scenario FMV at 5- and 10-year horizons |
| `fmv_bull_5yr / fmv_bear_5yr` | Bull / Bear FMV at 5 years |
| `contractual_5yr / 10yr` | Projected contractual rent (escalation clause) |
| `gap_direction` | WIDENING, NARROWING, FLIPPING, AT_MARKET, or STABLE |
| `recommendation` | RENEGOTIATE_NOW, LOCK_IN, or MONITOR |

In [ ]:
# ============================================================
# Section 6.1 — Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import os
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 110
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

# ---- Forecast configuration ----
CURRENT_YEAR  = 2026
HORIZON_YEARS = 10                    # forecast out to 2036
FORECAST_YEARS = np.arange(CURRENT_YEAR, CURRENT_YEAR + HORIZON_YEARS + 1)

# Scenario parameters (aligned with configs/model_config.yaml)
SCENARIOS = {
    'bull': {'cpi_growth': 0.035, 'pop_growth_mult': 1.3,
             'desc': 'High inflation, strong demand, tower scarcity'},
    'base': {'cpi_growth': 0.025, 'pop_growth_mult': 1.0,
             'desc': 'Trend continuation'},
    'bear': {'cpi_growth': 0.015, 'pop_growth_mult': 0.7,
             'desc': 'Low inflation, weak demand, tower buildout'},
}

print('All imports loaded successfully.')
print(f'Forecast horizon: {CURRENT_YEAR} \u2192 {CURRENT_YEAR + HORIZON_YEARS}')
print(f'Scenarios: {list(SCENARIOS.keys())}')
for s, p in SCENARIOS.items():
    print(f'  {s.upper():>5s}: CPI={p["cpi_growth"]*100:.1f}%, '
          f'PopMult={p["pop_growth_mult"]:.1f}x  \u2014 {p["desc"]}')

In [ ]:
# ============================================================
# 6.1b — Data Load (prefer processed predictions, fallback to synthetic)
# ============================================================
PREDICTIONS_PATH = os.path.join('..', 'data', 'processed', 'fmv_predictions.parquet')
SYNTHETIC_PATH   = os.path.join('..', 'data', 'synthetic', 'synthetic_5k_sites.parquet')

if os.path.exists(PREDICTIONS_PATH):
    df = pd.read_parquet(PREDICTIONS_PATH)
    DATA_SOURCE = 'fmv_predictions'
    print(f'\u2713 Loaded FMV predictions: {df.shape}')
elif os.path.exists(SYNTHETIC_PATH):
    df = pd.read_parquet(SYNTHETIC_PATH)
    DATA_SOURCE = 'synthetic_fallback'
    print(f'\u2717 Predictions not found. Loaded synthetic data (fallback): {df.shape}')
else:
    raise FileNotFoundError(
        'Neither fmv_predictions.parquet nor synthetic_5k_sites.parquet found.\n'
        'Run Notebook 03 or Simulation 00 first.'
    )

# ---- Quick inline pipeline if using synthetic fallback --------
if DATA_SOURCE == 'synthetic_fallback':
    print('\n--- Running quick inline FMV prediction (synthetic fallback) ---')
    if 'true_fmv' in df.columns:
        # Simulate model output: true_fmv + lognormal noise
        noise = np.random.lognormal(0, 0.08, len(df))
        df['predicted_fmv'] = (df['true_fmv'] * noise).round(2)
    elif 'de_escalated_base_rent' in df.columns:
        df['predicted_fmv'] = df['de_escalated_base_rent'].values
    else:
        raise KeyError('Cannot derive predicted_fmv: need true_fmv or de_escalated_base_rent.')

    if 'fmv_classification' not in df.columns:
        dev = (df['current_monthly_rent'] - df['predicted_fmv']) / df['predicted_fmv']
        df['fmv_classification'] = np.where(
            dev > 0.15, 'OVERPAYING',
            np.where(dev < -0.15, 'UNDERPAYING', 'AT_MARKET')
        )
    print(f'  predicted_fmv range: ${df["predicted_fmv"].min():,.0f} '
          f'\u2013 ${df["predicted_fmv"].max():,.0f}')

# ---- Validate required columns ---------------------------------
required_cols = ['site_id', 'metro_region', 'current_monthly_rent',
                 'escalation_rate', 'lease_age_years', 'predicted_fmv']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise KeyError(f'Missing required columns: {missing}')

# Derive lease_start_year if not present
if 'lease_start_year' not in df.columns:
    df['lease_start_year'] = (CURRENT_YEAR - df['lease_age_years']).astype(int)

print(f'\nDataset ready: {df.shape[0]:,} sites, {df.shape[1]} columns')
print(f'Metro regions: {df["metro_region"].nunique()}')
print(f'Data source:   {DATA_SOURCE}')
print(f'Lease age:     {df["lease_age_years"].min():.0f} \u2013 {df["lease_age_years"].max():.0f} years')
print(f'Escalation:    {df["escalation_rate"].min()*100:.1f}% \u2013 {df["escalation_rate"].max()*100:.1f}%')
df[['site_id', 'metro_region', 'current_monthly_rent', 'predicted_fmv',
    'escalation_rate', 'lease_age_years']].head()

---
## Section 6.2 — Construct Regional De-Escalated Rent Index

**Goal:** Build a time-series of *de-escalated* rents per metro region, stripping out contractual
escalation to reveal the underlying market signal.

**Method:**
1. For each site, back out **base rent at inception**: `base_rent = current_rent / (1 + esc)^age`
2. Simulate its annual rent path from `lease_start_year` to 2026:
   - `contractual_rent[t] = base_rent * (1 + escalation_rate)^(t - start_year)`
3. De-escalate: `de_escalated_rent[t] = contractual_rent[t] / cumulative_escalation[t]`
   - For non-amended sites this reduces to `base_rent` (constant)
4. For amended sites (`amendment_count > 0`), inject a **rent-reset jump** at the lease midpoint
5. Aggregate by `(metro_region, year)` using the **median** across sites

> **Note:** With synthetic data, de-escalated rent is approximately constant for most sites.
> With real payment histories, this index would capture richer market dynamics including
> genuine rent resets, buyout negotiations, and regional supply/demand shifts.

In [ ]:
# ============================================================
# 6.2a — Simulate site-level annual de-escalated rent paths
# ============================================================
HISTORY_START = 2001   # earliest plausible lease start in portfolio
ALL_YEARS = np.arange(HISTORY_START, CURRENT_YEAR + 1)

# Pre-compute base rent at inception for every site
# base_rent = current_rent / (1 + esc_rate)^age
cumulative_esc = (1 + df['escalation_rate'].values) ** df['lease_age_years'].values
base_rent = df['current_monthly_rent'].values / cumulative_esc

# Check for amendment_count column
has_amendments = 'amendment_count' in df.columns
if has_amendments:
    amendment_flags = df['amendment_count'].values > 0
    print(f'Sites with amendments: {amendment_flags.sum():,} '
          f'({amendment_flags.mean()*100:.1f}%)')
else:
    amendment_flags = np.zeros(len(df), dtype=bool)
    print('No amendment_count column found \u2014 all sites treated as non-amended.')

# Build long-form panel: (site_id, year, de_escalated_rent)
records = []
for i in range(len(df)):
    start_yr = int(df.iloc[i]['lease_start_year'])
    esc_rate = df.iloc[i]['escalation_rate']
    br       = base_rent[i]
    site     = df.iloc[i]['site_id']
    region   = df.iloc[i]['metro_region']

    for yr in range(max(start_yr, HISTORY_START), CURRENT_YEAR + 1):
        t = yr - start_yr
        contractual = br * (1 + esc_rate) ** t
        cum_esc_t   = (1 + esc_rate) ** t
        de_esc      = contractual / cum_esc_t      # = base_rent for non-amended

        # Inject amendment jump: after midpoint of lease the
        # de-escalated rent jumps by ~15% (rent-reset)
        if has_amendments and amendment_flags[i]:
            amendment_yr = start_yr + int(df.iloc[i]['lease_age_years'] * 0.5)
            if yr >= amendment_yr:
                de_esc *= 1.15     # 15% rent-reset bump

        records.append((site, region, yr, de_esc))

rent_panel = pd.DataFrame(records,
                           columns=['site_id', 'metro_region', 'year', 'de_escalated_rent'])

print(f'\nSite-level rent panel: {rent_panel.shape[0]:,} rows')
print(f'Year range: {rent_panel["year"].min()} \u2013 {rent_panel["year"].max()}')
print(f'Unique sites: {rent_panel["site_id"].nunique():,}')
rent_panel.head()

In [ ]:
# ============================================================
# 6.2b — Aggregate to regional de-escalated rent index
# ============================================================
regional_index = (
    rent_panel
    .groupby(['metro_region', 'year'])['de_escalated_rent']
    .agg(['median', 'mean', 'std', 'count'])
    .reset_index()
)
regional_index.columns = ['metro_region', 'year', 'median_de_esc_rent',
                           'mean_de_esc_rent', 'std_de_esc_rent', 'site_count']

# Filter to years with at least 5 contributing sites for statistical reliability
regional_index = regional_index[regional_index['site_count'] >= 5].copy()

print(f'Regional index: {regional_index.shape[0]:,} (region, year) pairs')
print(f'Regions:        {regional_index["metro_region"].nunique()}')
print(f'Median rent across all region-year pairs: '
      f'${regional_index["median_de_esc_rent"].median():,.0f}/mo')
print(f'\nSample (first available large metro):')

# Show sample for a well-populated region
_sample_region = (
    regional_index.groupby('metro_region')['site_count']
    .max().sort_values(ascending=False).index[0]
)
print(f'  Region: {_sample_region}')
regional_index[regional_index['metro_region'] == _sample_region].tail(8)

In [ ]:
# ============================================================
# 6.2c — Plot: De-escalated rent index for 5 sample regions
# ============================================================
# Pick 5 diverse regions (mix of high-growth, stable, rural)
top_regions = df['metro_region'].value_counts().head(15).index.tolist()

preferred = ['New York', 'Phoenix', 'Chicago', 'Austin', 'Rural Montana']
sample_regions = [r for r in preferred if r in top_regions]
for r in top_regions:
    if len(sample_regions) >= 5:
        break
    if r not in sample_regions:
        sample_regions.append(r)
sample_regions = sample_regions[:5]

fig, ax = plt.subplots(figsize=(12, 5))
palette = sns.color_palette('Set2', len(sample_regions))

for i, region in enumerate(sample_regions):
    mask = regional_index['metro_region'] == region
    sub  = regional_index[mask].sort_values('year')
    ax.plot(sub['year'], sub['median_de_esc_rent'], marker='o', markersize=3,
            label=region, color=palette[i], linewidth=2)

ax.set_xlabel('Year')
ax.set_ylabel('Median De-Escalated Rent ($/mo)')
ax.set_title('Section 6.2 \u2014 Regional De-Escalated Rent Index (5 Sample Regions)',
             fontsize=13)
ax.legend(frameon=True, fontsize=9)
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

print('\nInterpretation:')
print('  \u2022 Flat lines = stable underlying market rents (typical for synthetic data).')
print('  \u2022 Upward jumps = lease amendments / rent resets within that region.')
print('  \u2022 With real data, this index would show genuine supply-demand-driven trends.')

---
## Section 6.3A — Panel Regression Model

**Goal:** Estimate the relationship between de-escalated rents and macroeconomic drivers
using a **panel OLS regression with region fixed effects**.

**Model specification:**

$$\text{median\_de\_esc\_rent}_{r,t} = \alpha_r + \beta_1 \cdot \text{CPI}_t + \beta_2 \cdot \text{pop\_growth}_{r,t} + \beta_3 \cdot \text{unemployment}_t + \varepsilon_{r,t}$$

where $\alpha_r$ are region fixed effects.

**Macro variables** (synthetic for this demo; real pipeline would use FRED / Census APIs):
- **CPI Index:** starts at 100 in 2001, grows 2–3%/yr with stochastic noise
- **Population Growth:** per-region annual rate, calibrated to metro tier
- **Unemployment Rate:** national rate with business-cycle dynamics and recession shocks

> **Note:** With synthetic data, macro variables may show weak signal relative to region
> fixed effects.  With real FRED/Census data, CPI and population growth would typically
> be highly significant predictors of de-escalated rents.

In [ ]:
# ============================================================
# 6.3A-1 — Generate synthetic macro trajectories
# ============================================================
np.random.seed(123)

# ---- CPI index: start at 100 in 2001, grow ~2-3%/yr with noise ----
cpi_series = [100.0]
for yr in range(HISTORY_START + 1, CURRENT_YEAR + 1):
    annual_growth = np.random.uniform(0.015, 0.04)
    cpi_series.append(cpi_series[-1] * (1 + annual_growth))
cpi_df = pd.DataFrame({
    'year': np.arange(HISTORY_START, CURRENT_YEAR + 1),
    'cpi_index': cpi_series
})

# ---- National unemployment trajectory ----
unemp_base = 5.0
unemp_series = []
for yr in range(HISTORY_START, CURRENT_YEAR + 1):
    # Business cycle (~10-yr period) plus recession shocks
    cycle = 1.5 * np.sin(2 * np.pi * (yr - 2001) / 10)
    shock = 4.0 if yr in [2009, 2020] else 0.0  # GFC + COVID
    unemp_series.append(
        max(3.0, unemp_base + cycle + shock + np.random.normal(0, 0.3))
    )
unemp_df = pd.DataFrame({
    'year': np.arange(HISTORY_START, CURRENT_YEAR + 1),
    'unemployment_rate': unemp_series
})

# ---- Regional population growth (per-region, per-year) ----
regions = regional_index['metro_region'].unique()
pop_growth_records = []
for region in regions:
    if 'Rural' in region:
        base_growth = 0.002     # low / negative
    elif region in ['Phoenix', 'Austin', 'Dallas', 'Nashville', 'Charlotte']:
        base_growth = 0.018     # high-growth sunbelt
    elif region in ['New York', 'Chicago', 'Philadelphia']:
        base_growth = 0.005     # stable / slow
    else:
        base_growth = 0.010     # moderate

    for yr in range(HISTORY_START, CURRENT_YEAR + 1):
        growth = base_growth + np.random.normal(0, 0.003)
        pop_growth_records.append((region, yr, max(-0.01, growth)))

pop_growth_df = pd.DataFrame(pop_growth_records,
                              columns=['metro_region', 'year', 'population_growth'])

print('Synthetic macro variables generated:')
print(f'  CPI:          {cpi_df["cpi_index"].iloc[0]:.0f} (2001) \u2192 '
      f'{cpi_df["cpi_index"].iloc[-1]:.1f} ({CURRENT_YEAR})')
print(f'  Unemployment: {min(unemp_series):.1f}% \u2013 {max(unemp_series):.1f}%')
print(f'  Pop growth:   {len(pop_growth_records):,} region-year records')

# ---- Quick visualization of macro trajectories ----
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(cpi_df['year'], cpi_df['cpi_index'], color='steelblue', linewidth=2)
axes[0].set_title('CPI Index')
axes[0].set_ylabel('Index (2001=100)')

axes[1].plot(unemp_df['year'], unemp_df['unemployment_rate'],
             color='indianred', linewidth=2)
axes[1].set_title('Unemployment Rate (%)')
axes[1].set_ylabel('Rate (%)')

for region in ['Phoenix', 'New York', 'Rural Montana']:
    sub = pop_growth_df[pop_growth_df['metro_region'] == region]
    if len(sub) > 0:
        cumulative = (1 + sub['population_growth']).cumprod()
        axes[2].plot(sub['year'], cumulative, label=region, linewidth=1.5)
axes[2].set_title('Cumulative Population Growth')
axes[2].set_ylabel('Growth Factor (2001=1)')
axes[2].legend(fontsize=8)

for ax in axes:
    ax.set_xlabel('Year')
    ax.grid(True, alpha=0.3)

fig.suptitle('Section 6.3A \u2014 Synthetic Macro Variables', y=1.02, fontsize=13)
fig.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 6.3A-2 — Build panel dataset and fit OLS with region FE
# ============================================================
# Merge regional index with macro variables
panel = regional_index.merge(cpi_df, on='year', how='left')
panel = panel.merge(unemp_df, on='year', how='left')
panel = panel.merge(pop_growth_df, on=['metro_region', 'year'], how='left')
panel = panel.dropna(subset=['cpi_index', 'unemployment_rate', 'population_growth'])

# Create region dummies for fixed effects (drop_first for identification)
region_dummies = pd.get_dummies(panel['metro_region'], prefix='fe', drop_first=True)

# Build design matrix
X_vars = ['cpi_index', 'population_growth', 'unemployment_rate']
X = panel[X_vars].copy()
X = pd.concat([X, region_dummies], axis=1)
X = sm.add_constant(X)

y = panel['median_de_esc_rent']

# Fit OLS with heteroskedasticity-robust standard errors (HC1)
model = OLS(y, X).fit(cov_type='HC1')

print('=' * 70)
print('PANEL REGRESSION: De-Escalated Rent ~ Macro + Region Fixed Effects')
print('=' * 70)
print(f'\nObservations:  {model.nobs:.0f}')
print(f'R-squared:     {model.rsquared:.4f}')
print(f'Adj R-squared: {model.rsquared_adj:.4f}')
print(f'F-statistic:   {model.fvalue:.2f}  (p={model.f_pvalue:.2e})')

# Print macro variable coefficients (not all region FE dummies)
print(f'\n{"Variable":<25} {"Coeff":>10} {"Std Err":>10} {"t-stat":>10} {"p-value":>10}')
print('-' * 70)
for var in ['const'] + X_vars:
    coef = model.params[var]
    se   = model.bse[var]
    t    = model.tvalues[var]
    p    = model.pvalues[var]
    sig  = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else ''
    print(f'{var:<25} {coef:>10.4f} {se:>10.4f} {t:>10.3f} {p:>10.4f} {sig}')

n_fe = region_dummies.shape[1]
print(f'\n+ {n_fe} region fixed effects (not shown)')
print(f'\nNote: With synthetic data, macro variables may show weak signal.')
print(f'      With real FRED/Census data, CPI and population growth would be')
print(f'      highly significant predictors of de-escalated rents.')

In [ ]:
# ============================================================
# 6.3A-3 — Regression diagnostics: residual plots
# ============================================================
residuals = model.resid
fitted    = model.fittedvalues

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# (a) Residuals vs Fitted
axes[0].scatter(fitted, residuals, alpha=0.3, s=12, color='steelblue')
axes[0].axhline(y=0, color='red', linewidth=0.8, linestyle='--')
axes[0].set_xlabel('Fitted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Fitted')

# (b) Q-Q plot
sm.qqplot(residuals, line='45', ax=axes[1], markersize=3, alpha=0.4)
axes[1].set_title('Q-Q Plot of Residuals')

# (c) Residual distribution
axes[2].hist(residuals, bins=40, edgecolor='white', color='steelblue', alpha=0.8)
axes[2].axvline(x=0, color='red', linewidth=0.8, linestyle='--')
axes[2].set_xlabel('Residual')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Residual Distribution')

fig.suptitle('Section 6.3A \u2014 Panel Regression Diagnostics', y=1.02, fontsize=13)
fig.tight_layout()
plt.show()

# Summary stats on residuals
print(f'Residual mean:     {residuals.mean():.4f}')
print(f'Residual std:      {residuals.std():.2f}')
print(f'Residual skewness: {stats.skew(residuals):.3f}')
print(f'Residual kurtosis: {stats.kurtosis(residuals):.3f}')

---
## Section 6.3C — Scenario-Based FMV Projection (Main Output)

**Goal:** Project FMV for every site over the next 10 years under three macroeconomic scenarios.

### Scenarios

| Scenario | CPI Growth | Pop Growth Mult | Description |
|----------|------------|-----------------|-------------|
| **BULL** | 3.5%/yr | 1.3x base | High inflation, strong demand, tower scarcity |
| **BASE** | 2.5%/yr | 1.0x base | Trend continuation |
| **BEAR** | 1.5%/yr | 0.7x base | Low inflation, weak demand, tower buildout |

### Growth Rate Mapping

Each metro region is mapped to a **base FMV growth rate** reflecting its structural characteristics:

| Tier | Metros | Base Growth | Rationale |
|------|--------|-------------|----------|
| **High-Growth** | Phoenix, Austin, Dallas, Nashville, Charlotte, Las Vegas, San Antonio, Atlanta | 3.0% | Sunbelt migration, rapid development |
| **Stable** | New York, Chicago, Philadelphia, Boston, Minneapolis, Portland | 2.0% | Mature markets, slower demand growth |
| **Moderate** | Other urban metros | 2.5% | Mixed dynamics |
| **Rural** | All "Rural" regions | 1.5% | Limited demand drivers |

Scenario adjustments combine CPI ratio and population multiplier on top of the base rate:

$$\text{growth\_rate}_{\text{scenario}} = \text{region\_base} \times \frac{\text{CPI}_{\text{scenario}}}{\text{CPI}_{\text{base}}} \times (0.5 + 0.5 \times \text{pop\_mult})$$

In [ ]:
# ============================================================
# 6.3C-1 — Map metro regions to growth rate tiers
# ============================================================
HIGH_GROWTH_METROS = ['Phoenix', 'Austin', 'Dallas', 'Nashville', 'Charlotte',
                       'Las Vegas', 'San Antonio', 'Atlanta']
STABLE_METROS = ['New York', 'Chicago', 'Philadelphia', 'Boston',
                  'Minneapolis', 'Portland']


def get_region_base_growth(region: str) -> float:
    """Map metro region name to base annual FMV growth rate."""
    if 'Rural' in region:
        return 0.015
    elif region in HIGH_GROWTH_METROS:
        return 0.030
    elif region in STABLE_METROS:
        return 0.020
    else:
        return 0.025   # moderate


df['region_base_growth'] = df['metro_region'].apply(get_region_base_growth)

# Compute scenario-specific growth rates per site
for scenario, params in SCENARIOS.items():
    cpi_adj = params['cpi_growth'] / SCENARIOS['base']['cpi_growth']
    pop_adj = params['pop_growth_mult']
    df[f'growth_rate_{scenario}'] = (
        df['region_base_growth'] * cpi_adj * (0.5 + 0.5 * pop_adj)
    )

# ---- Display growth rate summary ----
print('Growth rate distribution by scenario:')
for scenario in SCENARIOS:
    col = f'growth_rate_{scenario}'
    print(f'  {scenario.upper():>5s}: mean={df[col].mean()*100:.2f}%, '
          f'min={df[col].min()*100:.2f}%, max={df[col].max()*100:.2f}%')

# Growth tier assignment counts
tier_map = {0.015: 'Rural (1.5%)', 0.020: 'Stable (2.0%)',
            0.025: 'Moderate (2.5%)', 0.030: 'High-Growth (3.0%)'}
tier_counts = df['region_base_growth'].map(tier_map).value_counts()
print(f'\nGrowth tier assignment:')
for tier, count in tier_counts.items():
    print(f'  {tier:<22s}: {count:>5,} sites ({count/len(df)*100:.1f}%)')

In [ ]:
# ============================================================
# 6.3C-2 — Project FMV and contractual rent for each site
# ============================================================
# FMV projection:       FMV[t] = predicted_fmv_today * (1 + growth_rate)^t
# Contractual rent:     rent[t] = current_rent * (1 + escalation_rate)^t

for t in range(1, HORIZON_YEARS + 1):
    yr = CURRENT_YEAR + t

    # FMV projections per scenario
    for scenario in SCENARIOS:
        rate = df[f'growth_rate_{scenario}'].values
        df[f'fmv_{scenario}_{yr}'] = (
            df['predicted_fmv'] * (1 + rate) ** t
        ).round(2)

    # Contractual rent projection
    df[f'contractual_{yr}'] = (
        df['current_monthly_rent'] * (1 + df['escalation_rate']) ** t
    ).round(2)

# ---- Extract key horizon columns for easy access ----
df['fmv_base_5yr']     = df[f'fmv_base_{CURRENT_YEAR + 5}']
df['fmv_base_10yr']    = df[f'fmv_base_{CURRENT_YEAR + 10}']
df['fmv_bull_5yr']     = df[f'fmv_bull_{CURRENT_YEAR + 5}']
df['fmv_bear_5yr']     = df[f'fmv_bear_{CURRENT_YEAR + 5}']
df['contractual_5yr']  = df[f'contractual_{CURRENT_YEAR + 5}']
df['contractual_10yr'] = df[f'contractual_{CURRENT_YEAR + 10}']

# ---- Gap analysis: contractual rent - FMV_base over time ----
df['gap_today'] = df['current_monthly_rent'] - df['predicted_fmv']
for t in range(1, HORIZON_YEARS + 1):
    yr = CURRENT_YEAR + t
    df[f'gap_{yr}'] = df[f'contractual_{yr}'] - df[f'fmv_base_{yr}']

# ---- Display sample site projection ----
print('Projections computed for all sites.')
print(f'\nSample site projection (row 0):')
site0 = df.iloc[0]
print(f'  Site:               {site0["site_id"]}  ({site0["metro_region"]})')
print(f'  FMV today:          ${site0["predicted_fmv"]:>10,.2f}')
print(f'  FMV base  5yr:      ${site0["fmv_base_5yr"]:>10,.2f}')
print(f'  FMV base 10yr:      ${site0["fmv_base_10yr"]:>10,.2f}')
print(f'  FMV bull  5yr:      ${site0["fmv_bull_5yr"]:>10,.2f}')
print(f'  FMV bear  5yr:      ${site0["fmv_bear_5yr"]:>10,.2f}')
print(f'  Contractual  5yr:   ${site0["contractual_5yr"]:>10,.2f}')
print(f'  Contractual 10yr:   ${site0["contractual_10yr"]:>10,.2f}')
print(f'  Gap today:          ${site0["gap_today"]:>+10,.2f}')
print(f'  Gap 5yr:            ${df.iloc[0][f"gap_{CURRENT_YEAR+5}"]:>+10,.2f}')
print(f'  Gap 10yr:           ${df.iloc[0][f"gap_{CURRENT_YEAR+10}"]:>+10,.2f}')

---
## Section 6.4 — Lock-In vs Renegotiate Decision Matrix

**Goal:** For every site, determine whether the lessee should **renegotiate now**, **lock in**
the current lease, or **monitor** for a future opportunity.

### Decision Logic

We classify each site based on the **direction** of the gap trajectory
(contractual rent − FMV under the base scenario):

| Pattern | Gap Today | Gap 5yr | Action | Rationale |
|---------|-----------|---------|--------|-----------|
| **WIDENING** | Positive & growing | Larger positive | **RENEGOTIATE_NOW** | Overpayment accelerating |
| **NARROWING** | Positive but shrinking | Smaller positive | **LOCK_IN** | Lease terms becoming favorable |
| **FLIPPING** | Positive → Negative | Sign change | **MONITOR** | Gap will flip; timing matters |
| **AT_MARKET** | |gap| < 10% of FMV | Small | **MONITOR** | Already near fair value |

Additionally, for underpaying sites:
- **WIDENING** underpayment (increasingly favorable) → **LOCK_IN**
- **NARROWING** underpayment → **MONITOR**

In [ ]:
# ============================================================
# 6.4a — Classify gap direction and assign recommendations
# ============================================================
gap_today = df['gap_today'].values
gap_5yr   = df[f'gap_{CURRENT_YEAR + 5}'].values
gap_10yr  = df[f'gap_{CURRENT_YEAR + 10}'].values

# Threshold: 10% of predicted FMV
threshold = 0.10 * df['predicted_fmv'].values

recommendations = []
gap_directions  = []
flip_years      = []

for i in range(len(df)):
    gt  = gap_today[i]
    g5  = gap_5yr[i]
    g10 = gap_10yr[i]
    thr = threshold[i]

    # ---- Find flip year (first year where gap changes sign) ----
    flip_yr = None
    for t in range(1, HORIZON_YEARS + 1):
        yr    = CURRENT_YEAR + t
        gap_t = df.iloc[i][f'gap_{yr}']
        if (gt > 0 and gap_t < 0) or (gt < 0 and gap_t > 0):
            flip_yr = yr
            break

    # ---- Classify ----
    if abs(gt) < thr and abs(g5) < thr:
        recommendations.append('MONITOR')
        gap_directions.append('AT_MARKET')
    elif flip_yr is not None:
        recommendations.append('MONITOR')
        gap_directions.append('FLIPPING')
    elif gt > thr and g5 > gt:
        recommendations.append('RENEGOTIATE_NOW')
        gap_directions.append('WIDENING')
    elif gt > thr and g5 <= gt:
        recommendations.append('LOCK_IN')
        gap_directions.append('NARROWING')
    elif gt < -thr and g5 < gt:
        recommendations.append('LOCK_IN')
        gap_directions.append('WIDENING')
    elif gt < -thr and g5 >= gt:
        recommendations.append('MONITOR')
        gap_directions.append('NARROWING')
    else:
        recommendations.append('MONITOR')
        gap_directions.append('STABLE')

    flip_years.append(flip_yr)

df['gap_direction']  = gap_directions
df['recommendation'] = recommendations
df['flip_year']      = flip_years

# ---- Print distributions ----
print('=' * 55)
print('DECISION MATRIX RESULTS')
print('=' * 55)

print(f'\nRecommendation Distribution:')
rec_counts = df['recommendation'].value_counts()
for rec, count in rec_counts.items():
    pct = count / len(df) * 100
    print(f'  {rec:<20s} {count:>5,} sites  ({pct:5.1f}%)')

print(f'\nGap Direction Distribution:')
dir_counts = df['gap_direction'].value_counts()
for direction, count in dir_counts.items():
    pct = count / len(df) * 100
    print(f'  {direction:<20s} {count:>5,} sites  ({pct:5.1f}%)')

# Sites with gap flip
flip_sites = df[df['flip_year'].notna()]
print(f'\nSites with gap flip within horizon: '
      f'{len(flip_sites):,} ({len(flip_sites)/len(df)*100:.1f}%)')
if len(flip_sites) > 0:
    print('  Flip year distribution:')
    print(flip_sites['flip_year'].value_counts().sort_index().to_string())

In [ ]:
# ============================================================
# 6.4b — Plot: 4 example site trajectories
# ============================================================

def find_example_site(df, direction, fallback_direction=None):
    """Find a representative example site with the given gap direction.
    Picks the site closest to the median absolute gap for that group."""
    candidates = df[df['gap_direction'] == direction]
    if len(candidates) == 0 and fallback_direction:
        candidates = df[df['gap_direction'] == fallback_direction]
    if len(candidates) == 0:
        return df.sample(1, random_state=42).index[0]
    med_gap = candidates['gap_today'].abs().median()
    return candidates.iloc[
        (candidates['gap_today'].abs() - med_gap).abs().argmin()
    ].name


examples = {
    'Widening Gap (RENEGOTIATE)':  find_example_site(df, 'WIDENING'),
    'Narrowing Gap (LOCK_IN)':     find_example_site(df, 'NARROWING'),
    'Flipping Gap (MONITOR)':      find_example_site(df, 'FLIPPING', 'STABLE'),
    'At-Market (MONITOR)':         find_example_site(df, 'AT_MARKET', 'STABLE'),
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()
forecast_years = np.arange(CURRENT_YEAR, CURRENT_YEAR + HORIZON_YEARS + 1)

for idx, (title, site_idx) in enumerate(examples.items()):
    ax   = axes[idx]
    site = df.loc[site_idx]

    # Contractual rent trajectory
    contractual_traj = [site['current_monthly_rent']]
    for t in range(1, HORIZON_YEARS + 1):
        contractual_traj.append(site[f'contractual_{CURRENT_YEAR + t}'])

    # FMV trajectories (3 scenarios)
    scenario_cfg = [
        ('bull', '#2ecc71', '--'),
        ('base', '#3498db', '-'),
        ('bear', '#e74c3c', '--'),
    ]
    for scenario, color, ls in scenario_cfg:
        fmv_traj = [site['predicted_fmv']]
        for t in range(1, HORIZON_YEARS + 1):
            fmv_traj.append(site[f'fmv_{scenario}_{CURRENT_YEAR + t}'])
        ax.plot(forecast_years, fmv_traj, color=color, linestyle=ls,
                linewidth=1.8, label=f'FMV ({scenario})', alpha=0.85)

    # Uncertainty band (bull–bear)
    bull_traj = [site['predicted_fmv']] + [
        site[f'fmv_bull_{CURRENT_YEAR+t}'] for t in range(1, HORIZON_YEARS+1)]
    bear_traj = [site['predicted_fmv']] + [
        site[f'fmv_bear_{CURRENT_YEAR+t}'] for t in range(1, HORIZON_YEARS+1)]
    ax.fill_between(forecast_years, bear_traj, bull_traj,
                    alpha=0.1, color='steelblue')

    # Contractual rent
    ax.plot(forecast_years, contractual_traj, color='#e67e22', linewidth=2.5,
            label='Contractual Rent', marker='o', markersize=3)

    ax.set_title(f'{title}\n({site["site_id"]}, {site["metro_region"]})',
                 fontsize=10)
    ax.set_xlabel('Year')
    ax.set_ylabel('$/month')
    ax.legend(fontsize=7, loc='upper left')
    ax.grid(True, alpha=0.3)

fig.suptitle(
    'Section 6.4 \u2014 Site-Level Forecast Trajectories '
    '(Contractual Rent vs FMV)',
    fontsize=14, y=1.01
)
fig.tight_layout()
plt.show()

print('\nPlot interpretation:')
print('  \u2022 Orange line:    contractual rent path (locked in by escalation clause)')
print('  \u2022 Blue solid:     base-case FMV forecast')
print('  \u2022 Green/Red dash: bull / bear scenario bounds')
print('  \u2022 Shaded area:    uncertainty band between bull and bear')

---
## Section 6.5 — Portfolio-Level Forecast Summary

**Goal:** Aggregate site-level forecasts into portfolio-wide metrics to support
strategic decision-making. Key questions answered:

1. What is the total portfolio FMV today vs. in 5 and 10 years?
2. How does total overpayment change over the horizon?
3. How many sites transition from overpaying to underpaying (or vice versa)?
4. Which metro regions carry the largest overpayment risk?

In [ ]:
# ============================================================
# 6.5a — Portfolio-level FMV and overpayment summary
# ============================================================
total_fmv_today  = df['predicted_fmv'].sum()
total_fmv_5yr    = df['fmv_base_5yr'].sum()
total_fmv_10yr   = df['fmv_base_10yr'].sum()

total_rent_today = df['current_monthly_rent'].sum()
total_rent_5yr   = df['contractual_5yr'].sum()
total_rent_10yr  = df['contractual_10yr'].sum()

total_gap_today  = total_rent_today - total_fmv_today
total_gap_5yr    = total_rent_5yr   - total_fmv_5yr
total_gap_10yr   = total_rent_10yr  - total_fmv_10yr

# Annual overpayment (monthly * 12)
annual_overpay_today = total_gap_today * 12
annual_overpay_5yr   = total_gap_5yr   * 12
annual_overpay_10yr  = total_gap_10yr  * 12

print('=' * 75)
print('PORTFOLIO-LEVEL FORECAST SUMMARY (BASE SCENARIO)')
print('=' * 75)
print(f'\n{"Metric":<35} {"Today":>14} {"5-Year":>14} {"10-Year":>14}')
print('-' * 80)
print(f'{"Total Monthly FMV":<35} ${total_fmv_today:>13,.0f} '
      f'${total_fmv_5yr:>13,.0f} ${total_fmv_10yr:>13,.0f}')
print(f'{"Total Monthly Rent":<35} ${total_rent_today:>13,.0f} '
      f'${total_rent_5yr:>13,.0f} ${total_rent_10yr:>13,.0f}')
print(f'{"Monthly Overpayment (gap)":<35} ${total_gap_today:>+13,.0f} '
      f'${total_gap_5yr:>+13,.0f} ${total_gap_10yr:>+13,.0f}')
print(f'{"Annual Overpayment":<35} ${annual_overpay_today:>+13,.0f} '
      f'${annual_overpay_5yr:>+13,.0f} ${annual_overpay_10yr:>+13,.0f}')
print(f'{"FMV Growth vs Today":<35} {"":>14} '
      f'{(total_fmv_5yr/total_fmv_today - 1)*100:>13.1f}% '
      f'{(total_fmv_10yr/total_fmv_today - 1)*100:>13.1f}%')

# ---- Transition analysis ----
overpaying_today = (df['gap_today'] > 0).sum()
overpaying_5yr   = (df[f'gap_{CURRENT_YEAR+5}'] > 0).sum()
overpaying_10yr  = (df[f'gap_{CURRENT_YEAR+10}'] > 0).sum()

over_to_under = ((df['gap_today'] > 0) & (df[f'gap_{CURRENT_YEAR+10}'] < 0)).sum()
under_to_over = ((df['gap_today'] < 0) & (df[f'gap_{CURRENT_YEAR+10}'] > 0)).sum()

print(f'\n{"Sites Overpaying":<35} {overpaying_today:>14,} '
      f'{overpaying_5yr:>14,} {overpaying_10yr:>14,}')
print(f'{"Sites Underpaying":<35} {len(df)-overpaying_today:>14,} '
      f'{len(df)-overpaying_5yr:>14,} {len(df)-overpaying_10yr:>14,}')
print(f'\nTransitions over 10 years:')
print(f'  Overpaying \u2192 Underpaying:  {over_to_under:>5,} sites')
print(f'  Underpaying \u2192 Overpaying:  {under_to_over:>5,} sites')

In [ ]:
# ============================================================
# 6.5b — Bar chart: overpayment by metro region (today vs 5yr)
# ============================================================
region_overpay = df.groupby('metro_region').agg(
    gap_today=('gap_today', 'sum'),
    gap_5yr=(f'gap_{CURRENT_YEAR+5}', 'sum'),
    site_count=('site_id', 'count')
).reset_index()

# Annualize
region_overpay['annual_overpay_today'] = region_overpay['gap_today'] * 12
region_overpay['annual_overpay_5yr']   = region_overpay['gap_5yr']   * 12

# Sort by today's overpayment descending
region_overpay = region_overpay.sort_values('annual_overpay_today', ascending=False)

top_n     = min(15, len(region_overpay))
plot_data = region_overpay.head(top_n)

fig, ax = plt.subplots(figsize=(14, 6))
x     = np.arange(len(plot_data))
width = 0.35

ax.bar(x - width/2, plot_data['annual_overpay_today'] / 1000,
       width, label='Today', color='#e74c3c', alpha=0.8)
ax.bar(x + width/2, plot_data['annual_overpay_5yr'] / 1000,
       width, label='5-Year Projection', color='#3498db', alpha=0.8)

ax.set_xlabel('Metro Region')
ax.set_ylabel('Annual Overpayment ($K)')
ax.set_title('Section 6.5 \u2014 Annual Overpayment by Metro Region: '
             'Today vs 5-Year Forecast (Base)', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(plot_data['metro_region'], rotation=45, ha='right', fontsize=8)
ax.legend()
ax.axhline(y=0, color='black', linewidth=0.5)
ax.grid(True, axis='y', alpha=0.3)

# Annotate site counts
for i, (_, row) in enumerate(plot_data.iterrows()):
    ax.annotate(f'n={row["site_count"]:.0f}', (i, 0),
                textcoords='offset points', xytext=(0, -15),
                ha='center', fontsize=7, color='gray')

fig.tight_layout()
plt.show()

print('Positive bars = portfolio overpaying in that region')
print('Negative bars = portfolio underpaying (favorable leases)')

In [ ]:
# ============================================================
# 6.5c — Portfolio overpayment trajectory over full horizon
# ============================================================
trajectory_years = ([CURRENT_YEAR] +
                    list(range(CURRENT_YEAR + 1, CURRENT_YEAR + HORIZON_YEARS + 1)))

# Compute total gap per year for each scenario
traj_data = {}
for scenario in SCENARIOS:
    gaps = [df['gap_today'].sum() * 12]          # year 0 (annualized)
    for t in range(1, HORIZON_YEARS + 1):
        yr    = CURRENT_YEAR + t
        gap_t = (df[f'contractual_{yr}'] - df[f'fmv_{scenario}_{yr}']).sum() * 12
        gaps.append(gap_t)
    traj_data[scenario] = gaps

fig, ax = plt.subplots(figsize=(12, 5))

colors = {'bull': '#2ecc71', 'base': '#3498db', 'bear': '#e74c3c'}
styles = {'bull': '--', 'base': '-', 'bear': '--'}

for scenario in SCENARIOS:
    vals = np.array(traj_data[scenario]) / 1e6   # convert to $M
    ax.plot(trajectory_years, vals, color=colors[scenario],
            linestyle=styles[scenario], linewidth=2.5,
            label=f'{scenario.upper()} scenario', marker='o', markersize=4)

# Bull–Bear uncertainty band
ax.fill_between(trajectory_years,
                np.array(traj_data['bear']) / 1e6,
                np.array(traj_data['bull']) / 1e6,
                alpha=0.1, color='steelblue')

ax.axhline(y=0, color='black', linewidth=0.8, linestyle=':')
ax.set_xlabel('Year')
ax.set_ylabel('Annual Portfolio Overpayment ($M)')
ax.set_title('Section 6.5 \u2014 Portfolio Overpayment Trajectory by Scenario',
             fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

print('Interpretation:')
print('  \u2022 Above zero: portfolio is net overpaying')
print('  \u2022 Below zero: portfolio is net underpaying (favorable)')
print('  \u2022 Shaded band: range of outcomes across bull / bear scenarios')

In [ ]:
# ============================================================
# 6.5d — Recommendation breakdown by metro region
# ============================================================
rec_by_region = pd.crosstab(
    df['metro_region'], df['recommendation'], margins=True
)

# Top 15 regions by site count
top_regions_list = df['metro_region'].value_counts().head(15).index.tolist()
rec_subset = rec_by_region.loc[
    [r for r in top_regions_list if r in rec_by_region.index]
]

fig, ax = plt.subplots(figsize=(14, 6))

rec_cols   = [c for c in ['RENEGOTIATE_NOW', 'LOCK_IN', 'MONITOR']
              if c in rec_subset.columns]
colors_map = {'RENEGOTIATE_NOW': '#e74c3c', 'LOCK_IN': '#2ecc71',
              'MONITOR': '#f39c12'}

bottom = np.zeros(len(rec_subset))
for col in rec_cols:
    vals = rec_subset[col].values
    ax.bar(range(len(rec_subset)), vals, bottom=bottom,
           label=col, color=colors_map.get(col, 'gray'), alpha=0.85)
    bottom += vals

ax.set_xlabel('Metro Region')
ax.set_ylabel('Number of Sites')
ax.set_title('Section 6.5 \u2014 Recommendation Distribution by Metro Region',
             fontsize=12)
ax.set_xticks(range(len(rec_subset)))
ax.set_xticklabels(rec_subset.index, rotation=45, ha='right', fontsize=8)
ax.legend()
ax.grid(True, axis='y', alpha=0.3)

fig.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 6.5e — Compact portfolio transition summary table
# ============================================================
summary_rows = []
for region in df['metro_region'].unique():
    sub = df[df['metro_region'] == region]
    n = len(sub)
    ot = (sub['gap_today'] > 0).sum()
    o5 = (sub[f'gap_{CURRENT_YEAR+5}'] > 0).sum()
    o10 = (sub[f'gap_{CURRENT_YEAR+10}'] > 0).sum()
    ot2u = ((sub['gap_today'] > 0) & (sub[f'gap_{CURRENT_YEAR+10}'] < 0)).sum()
    ut2o = ((sub['gap_today'] < 0) & (sub[f'gap_{CURRENT_YEAR+10}'] > 0)).sum()
    renegotiate = (sub['recommendation'] == 'RENEGOTIATE_NOW').sum()
    summary_rows.append({
        'metro_region': region, 'n_sites': n,
        'overpay_today': ot, 'overpay_5yr': o5, 'overpay_10yr': o10,
        'over_to_under': ot2u, 'under_to_over': ut2o,
        'renegotiate_now': renegotiate
    })

summary_table = (
    pd.DataFrame(summary_rows)
    .sort_values('renegotiate_now', ascending=False)
    .head(15)
    .reset_index(drop=True)
)

print('Portfolio Transition Summary (Top 15 Regions by Renegotiation Count):')
print()
print(summary_table.to_string(index=False))

---
## Section 6.6 — Save Outputs

Export the forecast results to `../data/processed/fmv_forecasts.parquet` with
the required columns for downstream consumption (dashboards, executive reports, etc.).

**Output schema:**
- `site_id` — unique identifier
- `predicted_fmv_today` — current FMV estimate
- `fmv_base_5yr / 10yr` — base scenario FMV at 5 and 10 years
- `fmv_bull_5yr / fmv_bear_5yr` — upside / downside FMV at 5 years
- `contractual_5yr / 10yr` — projected contractual rent
- `gap_direction` — classification of gap trajectory
- `recommendation` — RENEGOTIATE_NOW, LOCK_IN, or MONITOR
- Year-by-year columns for each scenario and gap

In [ ]:
# ============================================================
# 6.6 — Save forecast outputs
# ============================================================
output_cols = [
    'site_id',
    'metro_region',
    'predicted_fmv',                 # will be renamed to predicted_fmv_today
    'fmv_base_5yr',
    'fmv_base_10yr',
    'fmv_bull_5yr',
    'fmv_bear_5yr',
    'contractual_5yr',
    'contractual_10yr',
    'gap_today',
    'gap_direction',
    'recommendation',
    'flip_year',
    'region_base_growth',
    'growth_rate_bull',
    'growth_rate_base',
    'growth_rate_bear',
]

# Add all year-by-year forecast columns
for t in range(1, HORIZON_YEARS + 1):
    yr = CURRENT_YEAR + t
    for scenario in SCENARIOS:
        col = f'fmv_{scenario}_{yr}'
        if col in df.columns and col not in output_cols:
            output_cols.append(col)
    for prefix in ['contractual', 'gap']:
        col = f'{prefix}_{yr}'
        if col in df.columns and col not in output_cols:
            output_cols.append(col)

# Filter to columns actually present
output_cols = [c for c in output_cols if c in df.columns]

forecast_df = df[output_cols].copy()
forecast_df = forecast_df.rename(columns={'predicted_fmv': 'predicted_fmv_today'})

# Save
output_path = os.path.join('..', 'data', 'processed', 'fmv_forecasts.parquet')
os.makedirs(os.path.dirname(output_path), exist_ok=True)
forecast_df.to_parquet(output_path, index=False)

print(f'Forecast results saved to: {output_path}')
print(f'Shape: {forecast_df.shape}')
print(f'\nColumns ({len(forecast_df.columns)}):')
for i, col in enumerate(forecast_df.columns):
    print(f'  {i+1:>3}. {col}')

print(f'\n--- Sample output (first 5 rows, key columns) ---')
forecast_df[['predicted_fmv_today', 'fmv_base_5yr', 'fmv_base_10yr',
              'contractual_5yr', 'gap_direction', 'recommendation']].head()

---
## Summary

This notebook completed **Stage 6** of the FMV pipeline:

1. **De-Escalated Rent Index** (Section 6.2) — Stripped contractual escalation from
   historical rents to reveal underlying market signals per region.  For amended sites,
   a 15% rent-reset jump is visible in the index.

2. **Panel Regression** (Section 6.3A) — Estimated macro-level drivers (CPI, population
   growth, unemployment) of de-escalated rents with region fixed effects.  Residual
   diagnostics confirm model adequacy.

3. **Scenario Projections** (Section 6.3C) — Projected FMV under Bull, Base, and Bear
   scenarios for every site over a 10-year horizon, with region-specific growth rates
   mapped to four metro tiers (High-Growth, Stable, Moderate, Rural).

4. **Decision Matrix** (Section 6.4) — Classified every site into **RENEGOTIATE_NOW**,
   **LOCK_IN**, or **MONITOR** based on the trajectory of the contractual-rent-vs-FMV gap.
   Identified flip years where the gap changes sign.

5. **Portfolio Summary** (Section 6.5) — Aggregated forecasts to portfolio level, showing:
   - Total FMV and overpayment today vs. 5 and 10 years
   - Overpayment trajectory under all three scenarios
   - Transition counts (overpaying \u2192 underpaying and vice versa)
   - Regional recommendation breakdown

### Key outputs saved
- `../data/processed/fmv_forecasts.parquet` — site-level forecasts with all scenarios and recommendations

### Next steps
- Integrate with real macro data (FRED CPI, Census population) for production forecasts
- Refine growth rate mapping with historical comp transaction data
- Build interactive dashboard for scenario exploration
- Implement Monte Carlo simulation for probabilistic confidence intervals